In [1]:
import pandas as pd
import re
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
from mlflow.tracking import MlflowClient


import traceback


/Users/josealmendarez/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

df_DST = pd.read_csv("../data/DST.csv")
df_K = pd.read_csv("../data/K.csv")
df_QB = pd.read_csv("../data/QB.csv")
df_RB = pd.read_csv("../data/RB.csv")
df_TE = pd.read_csv("../data/TE.csv")
df_WR = pd.read_csv("../data/WR.csv")

df_DST['Position'] = 'DST'
df_K['Position'] = 'K'
df_QB['Position'] = 'QB'
df_RB['Position'] = 'RB'
df_TE['Position'] = 'TE'
df_WR['Position'] = 'WR'

df = pd.concat([df_DST, df_K, df_QB, df_RB, df_TE, df_WR], ignore_index=True)

print(f"Total de filas: {len(df)}")
print(f"\nDistribución por posición:")
print(df['Position'].value_counts())
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nColumnas:")
print(df.columns.tolist())
print(f"\nInfo del DataFrame:")
print(df.info())

Total de filas: 938

Distribución por posición:
Position
WR     325
RB     208
TE     192
QB     121
K       58
DST     34
Name: count, dtype: int64

Primeras filas:
   Rank                      Player  SACK  INT   FR   FF  DEF TD  SFTY  \
0   1.0      Seattle Seahawks (SEA)  12.0  7.0  0.0  0.0     0.0   0.0   
1   2.0  Jacksonville Jaguars (JAC)   7.0  9.0  4.0  5.0     0.0   0.0   
2   3.0     Minnesota Vikings (MIN)  11.0  2.0  5.0  8.0     2.0   0.0   
3   4.0   Philadelphia Eagles (PHI)   5.0  3.0  2.0  4.0     0.0   0.0   
4   5.0         Detroit Lions (DET)  14.0  3.0  3.0  4.0     0.0   0.0   

   SPC TD    G  ...  TD  SACKS ATT.1 YDS.1  TD.1  FL  20+  TGT  REC  Y/R  
0     2.0  4.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
1     1.0  4.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
2     0.0  4.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
3     2.0  4.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
4     1.0  4.0

In [3]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/almendarez1002@gmail.com/FantasyDraft"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

In [15]:
# ---------- 1) Definir target ----------
TARGET = "FPTS"

# ---------- 2) Quitar columnas que NO deben ser features ----------
# - Identificadores y texto
id_like = ["Player", "Team"]  # agrega otras si las tienes (e.g., 'PlayerId')
# - Fugas de información (derivadas del target o rankings)
leak_like_patterns = [
    r"^FPTS\/G$",      # puntos por juego (deriva del target)
    r"rank",           # cualquier 'rank' o variantes
    r"tier",           # tiers si existieran
]
# Compilar regex para filtrar
leak_regex = re.compile("|".join(leak_like_patterns), flags=re.IGNORECASE)

drop_cols = set(id_like + [TARGET])
drop_cols.update([c for c in df.columns if leak_regex.search(str(c))])

# ---------- 3) Seleccionar columnas numéricas y categóricas ----------
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns
            if c not in drop_cols and c != TARGET]

# Asegurar que Position esté como categórica
cat_cols = ["Position"]

# ---------- 4) Imputación + OneHot para 'Position' ----------
numeric_transformer = SimpleImputer(strategy="constant", fill_value=0)
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

# ---------- 5) Ajustar transformador y generar X, y ----------
X = preprocessor.fit_transform(df)
y = df[TARGET].values

# ---------- 6) Obtener nombres de features transformadas ----------
ohe_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols)
feature_names = np.r_[num_cols, ohe_feature_names]

print(f"Total features numéricas: {len(num_cols)}")
print(f"Total features categóricas (one-hot): {len(ohe_feature_names)}")
print(f"Total de features finales: {len(feature_names)}")

# Vista rápida de las primeras 25 columnas transformadas
print("\nEjemplo de nombres de features resultantes:")
print(feature_names[:25])


Total features numéricas: 32
Total features categóricas (one-hot): 6
Total de features finales: 38

Ejemplo de nombres de features resultantes:
['SACK' 'INT' 'FR' 'FF' 'DEF TD' 'SFTY' 'SPC TD' 'G' 'FG' 'FGA' 'PCT' 'LG'
 '1-19' '20-29' '30-39' '40-49' '50+' 'XPT' 'XPA' 'CMP' 'ATT' 'Y/A' 'TD'
 'SACKS' 'ATT.1']


In [5]:
# ------------------ 0) Configuración ------------------
TARGET = "FPTS"

# ------------------ 1) Asegurar tipos numéricos y target limpio ------------------
df = df.copy()

# Forzar columnas numéricas a ser realmente numéricas cuando aplique (sin romper strings válidos)
# Aquí convertimos solo el TARGET explícitamente; el resto lo manejará el ColumnTransformer con imputación
df[TARGET] = pd.to_numeric(df[TARGET], errors="coerce")

# Quitar filas con FPTS NaN/inf
mask = np.isfinite(df[TARGET])
df = df.loc[mask].reset_index(drop=True)

print(f"Filas después de limpiar {TARGET}: {len(df)}")

Filas después de limpiar FPTS: 926


In [6]:

# ------------------ 2) Definir columnas a eliminar (no-features) ------------------
# Identificadores y texto que no deben entrar como features
id_like = ["Player", "Team"]  # si faltan, se manejan luego para imprimir
# Fugas de información: cualquier cosa derivada del target o rankings
leak_like_patterns = [
    r"^FPTS\/G$",   # puntos por juego (deriva del target)
    r"\brank\b",    # rank, Rank, RANK
    r"\btier\b",    # tier, Tier, TIER
]
leak_regex = re.compile("|".join(leak_like_patterns), flags=re.IGNORECASE)

drop_cols = set(id_like + [TARGET])
drop_cols.update([c for c in df.columns if leak_regex.search(str(c))])

In [7]:
# ------------------ 3) Columnas numéricas y categóricas ------------------
# Asegurar que Position exista
if "Position" not in df.columns:
    raise ValueError("No se encontró la columna 'Position' en el DataFrame.")

num_cols = [c for c in df.select_dtypes(include=[np.number]).columns
            if c not in drop_cols and c != TARGET]
cat_cols = ["Position"]

print(f"Total features numéricas (detectadas): {len(num_cols)}")
print(f"Total features categóricas (one-hot): {len(cat_cols)} → {cat_cols}")

Total features numéricas (detectadas): 32
Total features categóricas (one-hot): 1 → ['Position']


In [8]:
# ------------------ 4) Preprocesamiento ------------------
# Imputación numérica con 0 (robusto para ausencias por posición)
numeric_transformer = SimpleImputer(strategy="constant", fill_value=0)

# Para categórica: imputar más frecuente y luego OneHot (evita NaNs en OHE)
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

In [9]:
X_train_df, X_test_df, y_train, y_test = train_test_split(
    df.drop(columns=[TARGET]), df[TARGET], test_size=0.2, random_state=42
)

In [24]:
# ------------------ CÓDIGO COMPLETO ADAPTADO PARA DATABRICKS COMMUNITY ------------------

EXPERIMENT_NAME = "/Users/almendarez1002@gmail.com/FantasyDraft"
model_name = "FantasyDraft_Model"

mlflow.set_experiment(EXPERIMENT_NAME)
client = MlflowClient()

# ------------------ 2) Definir función objetivo Optuna ------------------
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-5, 0.5, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-5, 0.5, log=True),
        "random_state": 42,
        "n_jobs": -1,
    }

    with mlflow.start_run(nested=True):
        mlflow.log_params(params)
        model = XGBRegressor(**params)
        pipeline = Pipeline(steps=[
            ("preprocessor", preprocessor),
            ("model", model)
        ])
        pipeline.fit(X_train_df, y_train)
        y_pred = pipeline.predict(X_test_df)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
    return rmse

# ------------------ 3) ENTRENAMIENTO PRINCIPAL ------------------
print("\n🚀 INICIANDO ENTRENAMIENTO")
print("="*60)

with mlflow.start_run(run_name="xgboost_FantasyDraft_training") as run:
    print(f"📊 Run ID: {run.info.run_id}")

    # Optimización con Optuna
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=5)

    best_params = study.best_params
    mlflow.log_params({f"best_{k}": v for k, v in best_params.items()})
    mlflow.log_metric("best_rmse", study.best_value)

    print(f"\n🎯 Mejores parámetros encontrados:")
    for k, v in best_params.items():
        print(f"   {k}: {v}")

    # Entrenar modelo final
    best_model = XGBRegressor(**best_params)
    xgb_pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", best_model)
    ])
    xgb_pipeline.fit(X_train_df, y_train)

    # Evaluar
    y_pred = xgb_pipeline.predict(X_test_df)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    mlflow.log_metrics({"final_rmse": rmse, "final_r2": r2})

    print(f"\n📈 MÉTRICAS DEL MODELO:")
    print(f"   RMSE: {rmse:.4f}")
    print(f"   R²:   {r2:.4f}")

    # ------------------ 4) GUARDAR Y REGISTRAR MODELO ------------------
    print("\n" + "="*60)
    print("💾 GUARDANDO MODELO EN MODEL REGISTRY")
    print("="*60)

    try:
        # Registrar modelo directamente (funciona en Community Edition)
        model_info = mlflow.sklearn.log_model(
            sk_model=xgb_pipeline,
            artifact_path="best_model",
            input_example=X_train_df.head(1),
            registered_model_name=model_name  # Registro directo
        )

        model_uri = f"runs:/{run.info.run_id}/best_model"
        print(f"✅ Modelo guardado en: {model_uri}")
        print(f"✅ Registrado como: {model_name}")

        # Agregar tags personalizados al run (esto siempre funciona)
        mlflow.set_tags({
            "model_type": "XGBoost",
            "model_version": "latest",
            "rmse": f"{rmse:.6f}",
            "r2": f"{r2:.6f}",
            "training_date": pd.Timestamp.now().isoformat(),
        })

    except Exception as e:
        print(f"⚠️ Error al registrar: {type(e).__name__}: {e}")
        # Fallback: solo guardar como artefacto
        model_info = mlflow.sklearn.log_model(
            sk_model=xgb_pipeline,
            artifact_path="best_model",
            input_example=X_train_df.head(1)
        )
        print(f"✅ Modelo guardado como artefacto (sin registro)")

    # ------------------ 5) CHAMPION vs CHALLENGER (VERSIÓN SIMPLIFICADA) ------------------
    print("\n" + "="*60)
    print("⚔️ EVALUACIÓN CHAMPION vs CHALLENGER")
    print("="*60)

    try:
        # Buscar todas las versiones del modelo
        all_runs = mlflow.search_runs(
            experiment_ids=[run.info.experiment_id],
            filter_string=f"tags.mlflow.runName LIKE '%FantasyDraft%'",
            order_by=["metrics.final_rmse ASC"],
            max_results=10
        )

        if len(all_runs) > 0:
            print(f"\n📊 HISTORIAL DE RUNS (Top 5 por RMSE):")
            print("-"*60)

            for idx, run_row in all_runs.head(5).iterrows():
                run_id = run_row['run_id']
                run_rmse = run_row.get('metrics.final_rmse', float('nan'))
                run_r2 = run_row.get('metrics.final_r2', float('nan'))
                run_date = pd.to_datetime(run_row['start_time']).strftime('%Y-%m-%d %H:%M')

                # Marcar el mejor
                symbol = "🏆" if idx == all_runs.index[0] else "📦"
                is_current = "← ACTUAL" if run_id == run.info.run_id else ""

                print(f"{symbol} {run_date} | RMSE: {run_rmse:.4f} | R²: {run_r2:.4f} {is_current}")

            # Comparación con el mejor histórico
            best_historical_rmse = all_runs.iloc[0]['metrics.final_rmse']
            best_run_id = all_runs.iloc[0]['run_id']

            print("\n" + "-"*60)
            if run.info.run_id == best_run_id:
                print("✅ 🏆 ESTE ES EL MEJOR MODELO HASTA AHORA")
                print(f"   RMSE: {rmse:.4f}")
            else:
                mejora = ((best_historical_rmse - rmse) / best_historical_rmse * 100)
                if mejora > 0:
                    print(f"✅ 🏆 NUEVO CHAMPION - Mejora: {mejora:.2f}%")
                    print(f"   Anterior mejor RMSE: {best_historical_rmse:.4f}")
                    print(f"   Nuevo RMSE: {rmse:.4f}")
                else:
                    print(f"🤜 CHALLENGER - No supera al champion")
                    print(f"   Champion RMSE: {best_historical_rmse:.4f}")
                    print(f"   Este modelo RMSE: {rmse:.4f}")
                    print(f"   Diferencia: {abs(mejora):.2f}% peor")
        else:
            print("ℹ️ Este es tu primer modelo registrado 🎉")

    except Exception as e:
        print(f"⚠️ No se pudo comparar con modelos anteriores: {type(e).__name__}")

    # ------------------ 6) RESUMEN FINAL ------------------
    print("\n" + "="*60)
    print("📋 RESUMEN DEL ENTRENAMIENTO")
    print("="*60)
    print(f"✅ Modelo: {model_name}")
    print(f"✅ Run ID: {run.info.run_id}")
    print(f"✅ RMSE: {rmse:.4f}")
    print(f"✅ R²: {r2:.4f}")
    print(f"✅ N° trials Optuna: 2")
    print(f"\n💡 Para usar el modelo:")
    print(f"   model_uri = 'runs:/{run.info.run_id}/best_model'")
    print(f"   loaded_model = mlflow.sklearn.load_model(model_uri)")
    print("="*60)

print("\n✅ PROCESO COMPLETADO")


🚀 INICIANDO ENTRENAMIENTO


[I 2025-11-11 22:40:04,147] A new study created in memory with name: no-name-65edf82c-7d79-4970-9606-d21bea0d1124


📊 Run ID: 9ec626ed40f24dfaa699e933ec509a3a


/Users/josealmendarez/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2025/11/11 22:40:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run valuable-hawk-586 at: https://dbc-88c60422-85f9.cloud.databricks.com/ml/experiments/1953745723580304/runs/82931dea4cfd4cb4b461a8002c606a53.
2025/11/11 22:40:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dbc-88c60422-85f9.cloud.databricks.com/ml/experiments/1953745723580304.
[I 2025-11-11 22:40:06,535] Trial 0 finished with value: 11.512634950822783 and parameters: {'n_estimators': 348, 'learning_rate': 0.002255085180715273, 'max_depth': 3, 'subsample': 0.7831255020013567, 'colsample_bytree': 0.8938812759492386, 'reg_alpha': 0.01347133168764303, 'reg_lambda': 0.0022676461415077247}. Best is trial


🎯 Mejores parámetros encontrados:
   n_estimators: 307
   learning_rate: 0.05821962247622297
   max_depth: 5
   subsample: 0.8989124284856352
   colsample_bytree: 0.7267042010375668
   reg_alpha: 0.01683322007933452
   reg_lambda: 0.3484704509524211


...

⚠️ Error al registrar: AttributeError: 'PermissionDenied' object has no attribute 'message'


Uploading artifacts: 100%|██████████| 7/7 [00:01<00:00,  6.19it/s]


✅ Modelo guardado como artefacto (sin registro)

⚔️ EVALUACIÓN CHAMPION vs CHALLENGER

📊 HISTORIAL DE RUNS (Top 5 por RMSE):
------------------------------------------------------------
🏆 2025-11-12 04:38 | RMSE: 4.1733 | R²: 0.9452 
📦 2025-11-12 04:40 | RMSE: 4.3437 | R²: 0.9407 ← ACTUAL
📦 2025-11-12 04:36 | RMSE: 4.7366 | R²: 0.9294 
📦 2025-11-12 04:31 | RMSE: nan | R²: nan 
📦 2025-11-12 04:28 | RMSE: nan | R²: nan 

------------------------------------------------------------
🤜 CHALLENGER - No supera al champion
   Champion RMSE: 4.1733
   Este modelo RMSE: 4.3437
   Diferencia: 4.08% peor

📋 RESUMEN DEL ENTRENAMIENTO
✅ Modelo: FantasyDraft_Model
✅ Run ID: 9ec626ed40f24dfaa699e933ec509a3a
✅ RMSE: 4.3437
✅ R²: 0.9407
✅ N° trials Optuna: 2

💡 Para usar el modelo:
   model_uri = 'runs:/9ec626ed40f24dfaa699e933ec509a3a/best_model'
   loaded_model = mlflow.sklearn.load_model(model_uri)


2025/11/11 22:40:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run xgboost_FantasyDraft_training at: https://dbc-88c60422-85f9.cloud.databricks.com/ml/experiments/1953745723580304/runs/9ec626ed40f24dfaa699e933ec509a3a.
2025/11/11 22:40:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dbc-88c60422-85f9.cloud.databricks.com/ml/experiments/1953745723580304.



✅ PROCESO COMPLETADO


In [13]:
# ------------------ 8) Importancias de características ------------------
# Recuperar nombres de columnas transformadas
num_cols_fitted = preprocessor.transformers_[0][2]
cat_cols_fitted = preprocessor.transformers_[1][2]
ohe_names = xgb_pipeline.named_steps["preprocessor"] \
                        .named_transformers_["cat"] \
                        .named_steps["ohe"] \
                        .get_feature_names_out(cat_cols_fitted)
feature_names_fitted = np.r_[num_cols_fitted, ohe_names]

importances = xgb_pipeline.named_steps["model"].feature_importances_
feat_imp = pd.DataFrame({"Feature": feature_names_fitted, "Importance": importances}) \
            .sort_values("Importance", ascending=False)

print("\n--- Top 15 Features más importantes ---")
print(feat_imp.head(15).reset_index(drop=True))



--- Top 15 Features más importantes ---
         Feature  Importance
0   Position_DST    0.311815
1             TD    0.261416
2            ATT    0.074204
3            CMP    0.062524
4             LG    0.042492
5              G    0.042059
6            REC    0.036152
7    Position_RB    0.022509
8            FGA    0.021670
9           SACK    0.018711
10            FG    0.018016
11         ATT.1    0.016477
12           INT    0.012113
13         YDS.1    0.010698
14          TD.1    0.007944


In [ ]:
# ------------------ 9) Predicciones completas y Top 10 Jugadores ------------------
df_pred = df.copy()

# Crear columna Team si viene con alias común
if "Team" not in df_pred.columns:
    for alt in ["Tm", "TEAM", "TeamAbbrev", "Franchise", "Club"]:
        if alt in df_pred.columns:
            df_pred["Team"] = df_pred[alt]
            break

# Crear columna Player si viene con alias
if "Player" not in df_pred.columns:
    for alt in ["Name", "PLAYER", "PlayerName"]:
        if alt in df_pred.columns:
            df_pred["Player"] = df_pred[alt]
            break

# Predicción sobre todo el dataset limpio
df_pred["Pred_FPTS"] = xgb_pipeline.predict(df.drop(columns=[TARGET]))

# Top 10
top_pred = df_pred.sort_values("Pred_FPTS", ascending=False).head(20)

# Seleccionar columnas disponibles para imprimir sin romper
display_cols = ["Player", "Position", "Team", "Pred_FPTS", "FPTS"]
available_cols = [c for c in display_cols if c in top_pred.columns]

print("\n--- Top 10 Jugadores Predichos por FPTS ---")
print(top_pred[available_cols].reset_index(drop=True))


--- Top 10 Jugadores Predichos por FPTS ---
                    Player Position  Pred_FPTS  FPTS
0         Josh Allen (BUF)       QB  98.937622  99.5
1      Lamar Jackson (BAL)       QB  93.878510  94.4
2  Patrick Mahomes II (KC)       QB  89.397346  89.6
3          Drake Maye (NE)       QB  86.014954  85.5
4        Jalen Hurts (PHI)       QB  84.346504  84.2
5     Caleb Williams (CHI)       QB  84.308907  84.1
6       Daniel Jones (IND)       QB  80.052292  80.5
7      Baker Mayfield (TB)       QB  79.908073  80.1
8     James Cook III (BUF)       RB  78.157944  79.0
9         Jordan Love (GB)       QB  75.189911  75.2
